<a href="https://colab.research.google.com/github/AditiLORenzo/Aditiprojects/blob/main/Netflix_recommendation_engine_using_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

loading the library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.sparse import csr_matrix
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
!pip install surprise

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
df1 = pd.read_csv('/content/combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])


In [ ]:
df1

In [ ]:
df1['Rating']=df1['Rating'].astype(float)

In [ ]:
print(f"Data shape {df1.shape}")
print("-Dataset Example-")
print(df1.iloc[::50000,:])

In [ ]:
df1.index=np.arange(0,len(df1))


In [ ]:
df1

In [ ]:
p=df1.groupby('Rating')['Rating'].agg(['count'])

In [ ]:
p

In [ ]:
p.iloc[2][0]

In [ ]:
#movie count that were not rated
movie_count=df1.isnull().sum()[1]

In [ ]:
df1.isnull().sum()

In [ ]:
#customer count
customer_count=df1['Cust_Id'].nunique()-movie_count

In [ ]:
#rating count
rating_count=df1['Cust_Id'].count()-movie_count

In [ ]:
df1['Rating'].count()-movie_count

In [ ]:
rating_count

##Data viewing

In [ ]:
ax=p.plot(kind='barh',legend=False,figsize=(15,10))
plt.title(f'Total pool:Movies{movie_count},Customer_count{customer_count},Ratings_given{rating_count}')
plt.axis('off')
for i in range(1,6):
    ax.text(p.iloc[i-1][0]/4, i-1, 'Rating {}: {:.0f}%'.format(i,p.iloc[i-1][0]*100 / p.sum()[0]), color = 'white', weight = 'bold')


##Data cleaning

In [ ]:
df_nan=pd.DataFrame(pd.isnull(df1.Rating))

In [ ]:
df_nan

In [ ]:
df_nan=df_nan[df_nan['Rating']==True]
df_nan

In [ ]:
df_nan=df_nan.reset_index()

In [ ]:
df_nan

In [ ]:
df=df1.reset_index()
df

In [ ]:
movie_np=[]
mn_id=1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), mn_id)
    movie_np = np.append(movie_np, temp)
    mn_id += 1

last_record=np.full((1,len(df1)-df_nan.iloc[-1,0]),mn_id)
movie_np=np.append(movie_np,last_record)

In [ ]:
print(f'movie_numpy{movie_np}')
print(f'length{len(movie_np)}')

In [ ]:
movie_np

In [ ]:
#removing null values from rating column
df1=df1[pd.notnull(df['Rating'])]

In [ ]:
#df = df[pd.notnull(df['Rating'])]
#df1['Movie_Id'] = movie_np.astype(int)
Cust_Id=df1['Cust_Id']
df1['Movie_Id'] = pd.Series(movie_np).astype(int)
#df1['Cust_Id'] = pd.Series(Cust_Id).astype(int)

In [ ]:
df1

In [ ]:
df1['Cust_Id']=df1['Cust_Id'].astype(int)

In [ ]:
df1

In [ ]:
print(f'-dataset-')
print(f'our datase{df1.iloc[::5,:]}')

In [ ]:
#for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
 #   print(i-j-1)


In [ ]:
#df_nan[''][1]-df_nan['index'][0]

In [ ]:
df_nan.iloc[-1,0]

In [ ]:
for i in range(1,6):
    print(p.iloc[i-1][0]/4,i-1)

##Data_slicing

In [ ]:
#df1.iloc[80000]

In [ ]:
df1

In [ ]:
f = ['count','mean']

df_movie_summary = df1.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index




In [ ]:
df_movie_summary['count']

In [ ]:
round(df_movie_summary['count'].quantile(0.7),0)#38.5% of all 55 numbers are lesser than 1307

In [ ]:
movie_benchmark

In [ ]:
#calculating customer summary
f=['count','mean']
df_cus_summary=df1.groupby('Cust_Id')['Rating'].agg(f)



In [ ]:
df_cus_summary

In [ ]:
df_cus_summary.index=df_cus_summary.index.map(int)

In [ ]:
df_cus_summary

In [ ]:
cus_bench_mark=round(df_cus_summary['count'].quantile(0.7),0)


In [ ]:
drop_cus_list=df_cus_summary[df_cus_summary['count']<cus_bench_mark].index

In [ ]:
print(f'movie minimum times of review{movie_benchmark}')
print(f'customer minimum times of review{cus_bench_mark}')

## ##trim down the data

In [ ]:
print(f'shape of df1 before{df1.shape}')
df1=df1[~df1['Movie_Id'].isin(drop_movie_list)]
df1=df1[~df1['Cust_Id'].isin([drop_cus_list])]
#print(f'trimmed movie list shape{df1.shape()}')

In [ ]:
print(f'trimmed movie list shape{df1.shape}')

In [ ]:
#pivot table
df_p=pd.pivot_table(df1,values='Rating',index='Cust_Id',columns='Movie_Id')
print(df_p.shape)

In [ ]:
df_p

##Data Mapping

In [ ]:
df_title=pd.read_csv('/content/movie_titles.csv',encoding="ISO-8859-1",header=None,names=['Movie_Id','Year','Name'])
df_title.set_index('Movie_Id',inplace=True)
print(df_title)

##Recommendation through collabrative filtering

In [ ]:
reader=Reader()
data=Dataset.load_from_df(df1[['Cust_Id','Movie_Id','Rating']][:],reader)
svd=SVD()
cross_validate(svd,data,measures=['RMSE','MAE'])

In [ ]:
df1.columns

In [ ]:
df1

In [ ]:
df_4000=df1[(df1[(df1['Cust_Id']==697)&df1['Rating']==3])]
df_4000=df_4000.set_index('Movie_Id')

In [ ]:
df_4000=df_4000.join(df_title)['Name']

In [ ]:
df_4000

In [ ]:
user=df_title.copy()
user=user.reset_index()
user=user[~user['Movie_Id'].isin(drop_movie_list)]
data=Dataset.load_from_df(df1[['Cust_Id','Movie_Id','Rating']],reader)

In [ ]:
trainset=data.build_full_trainset()

In [ ]:
svd.fit(trainset)

In [ ]:
user['Estimate_score']=user['Movie_Id'].apply(lambda x:svd.predict(657,x).est)
user=user.drop(['Movie_Id'],axis=1)
user=user.sort_values('Estimate_score',ascending=False)

In [ ]:
print(user.head(10))